# BBVA data

## Parameters

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Contests/Datathon_202109/data/datos'
CP_FILE = 'Canarias_CP.csv'
FOLDER_BBVA = 'CM01_3'

## Import

In [2]:
from mariachis.bbva import BbvaClass

bbva = BbvaClass(BASE_DIR, CP_FILE, FOLDER_BBVA)
print(bbva)

Folder path:	/Users/efraflores/Desktop/EF/Contests/Datathon_202109/data/datos/CM01_3
With 102 files about average amount and 103 about cards


## Transform

In [3]:
df, pipe_obj = bbva.full_pipeline(cluster_kwargs={'n_clusters':7, 'kmeans':False}, index=['zipcode'], values=['avg_amount','cards'], aggfunc=['count','sum','mean','median','min','max'], fill_value=0)
df.head(2)

,zipcode,count_avg_amount,count_cards,max_avg_amount,max_cards,mean_avg_amount,mean_cards,median_avg_amount,median_cards,min_avg_amount,...,place,state,state_code,province,province_code,comunity,comunity_code,lat,lng,accuracy
0,35001,671,671,162.08,43,25.619091,14.931446,23.30,14.0,5.63,...,Las Palmas De Gran Canaria,Canarias,CN,Las Palmas,GC,Las Palmas de Gran Canaria,35016.0,28.0997,-15.4134,1
1,35002,720,720,163.44,151,28.271667,69.365278,26.25,72.0,11.35,...,Las Palmas De Gran Canaria,Canarias,CN,Las Palmas,GC,Las Palmas de Gran Canaria,35016.0,28.0997,-15.4134,1


## Profiles

In [8]:
from mariachis.utils import profiles

prof = profiles(df[['cluster','province','comunity']].join(df.filter(regex='sum|mean|min|max')))

for var in prof.values():
    display(var.fillna(0).style.format("{:.0f}").background_gradient('Blues'))

,max_avg_amount,max_cards,mean_avg_amount,mean_cards,min_avg_amount,min_cards,sum_avg_amount,sum_cards
cluster,,,,,,,,
A,287,28,53,9,9,3,13615,4105
B,176,438,40,195,7,3,27209,133778
C,152,82,28,38,6,3,19291,26626
D,35,3,30,3,27,3,57,6
E,126,5,102,4,77,3,203,8
F,142,347,37,115,10,4,26908,83626
G,194,15,26,6,6,3,5760,1620


## Export

In [5]:
bbva.export_bbva(df)

Exported succesfully!
File:	Finished_BBVA.csv
Path:	/Users/efraflores/Desktop/EF/Contests/Datathon_202109/data/datos
